# Titanic survival prediction using decision tree

updated on 2021-10-17

my very first kaggle submission

In [ ]:
#0.76794

In [1]:
#import library and data 

import pandas as pd
from sklearn import preprocessing as p
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

train = pd.read_csv('D:/data/titanic/train.csv')
test = pd.read_csv('D:/data/titanic/test.csv')
submission = pd.read_csv('D:/data/titanic/gender_submission.csv')

In [2]:
train.info() #show columns and # of null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
train['Embarked'].value_counts() #what values are in embarked columns

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [6]:
#filling null values

train['Age'].fillna(train['Age'].mean(), inplace=True)
test['Age'].fillna(test['Age'].mean(), inplace=True)

train['Fare'].fillna(train['Fare'].mean(), inplace=True)
test['Fare'].fillna(test['Fare'].mean(), inplace=True)

train['Embarked'].fillna('S', inplace=True)
test['Embarked'].fillna('S', inplace=True)

In [7]:
#dropping columns that are not needed
features = ['Name', 'Cabin', 'PassengerId', 'Ticket']

train.drop(features, axis=1, inplace=True)
test.drop(features, axis=1, inplace=True)

In [8]:
#label encoding (transitioning words into numbers)
def encode(df):
    features = ['Sex', 'Embarked']
    for f in features:
        le = p.LabelEncoder()
        le = le.fit(df[f])
        df[f] = le.transform(df[f])
        
    return df

train = encode(train)
test = encode(test)

In [9]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [10]:
#check number of null values left
test.isnull().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [11]:
#split X and y
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'] 
label = ['Survived']

y_train = train[label]
X_train = train[features]
X_test = test[features]

In [12]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,2
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,2
3,1,0,35.0,1,0,53.1000,2
4,3,1,35.0,0,0,8.0500,2


In [13]:
y_train.head()

,Survived
0,0
1,1
2,1
3,1
4,0


In [14]:
#define model and fit
dt_clf= DecisionTreeClassifier(random_state=23)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=23)

In [15]:
#tune parameters using GridSearchCV
params = {
    'max_depth' : [2, 4, 6, 8, 10],
    'min_samples_split' : [2, 4, 6, 8, 10],
    'min_samples_leaf' : [2, 4, 6, 6, 10]
}

grid_dclf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_dclf.fit(X_train, y_train)

best_dclf = grid_dclf.best_estimator_
pred = best_dclf.predict(X_test)
submission['Survived'] = pred
submission.to_csv('D:/data/titanic/final.csv', index=False)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
